In [2]:
import pandas as pd
import numpy as np

In [1]:
def load_dataset2(self, csv_file, count=0, skip=0):
        """Подготовка обучающей выборки (x,y). Тип x и y - numpy.ndarray.
        Аргументы:
        csv_file - файл с ценами формата csv с колонками: 'date','time','open','high','low','close','tickvol','vol','spread'
        count - количество используемых примеров датасета
        skip - сдвиг относительно конца файла
        Возврат:
        x, y - размеченные данные типа numpy.array, сохранные на диск файл dataset_file"""
        # stride не менять, должно быть =1
        stride = 1  # шаг "нарезки" входных данных
        in_shape = self.datainfo.input_shape
        out_shape = self.datainfo.output_shape
        forward = self.datainfo.future
        in_size = self.datainfo._in_size()
        out_size = self.datainfo._out_size()  # out_shape[0]
        if not path.exists(csv_file):
            print('Отсутствует файл "' + csv_file + '"\nЗагрузка данных неуспешна')
            return None, None
        print("Чтение файла", csv_file, "и создание обучающей выборки")
        data = pd.read_csv(
            csv_file,
            sep="\t",
            header=0,
            dtype={
                "open": np.float32,
                "close": np.float32,
                "tickvol": np.float32,
                "vol": np.float32,
            },
            names=[
                "date",
                "time",
                "open",
                "high",
                "low",
                "close",
                "tickvol",
                "vol",
                "spread",
            ],
        )
        if skip == 0 and count == 0:
            open_rates = data["open"]
            vol_rates = data["tickvol"]
        elif skip == 0:
            open_rates = data["open"][-count:]
            vol_rates = data["tickvol"][-count:]
        elif count == 0:
            open_rates = data["open"][:-skip]
            vol_rates = data["tickvol"][:-skip]
        else:
            open_rates = data["open"][-count - skip : -skip]
            vol_rates = data["tickvol"][-count - skip : -skip]
        # объемы
        volumes = np.nan_to_num(np.array(vol_rates))
        volumes_strided = roll(opens[:-forward], in_size, stride)
        # цены
        opens = np.nan_to_num(np.diff(np.array(open_rates), n=1), posinf=8, neginf=-8)
        opens_strided = roll(opens[:-forward], in_size, stride)
        forward_values = roll(opens[in_size:], forward, stride).sum(axis=1)
        opens_std = opens.std()
        # open_mean = np.mean(opens)
        forward_values_std = forward_values.std()
        self.datainfo.x_std = float(opens_std)
        self.datainfo.y_std = float(forward_values_std)
        self.datainfo.save(self.name + ".cfg")

        x = np.reshape(opens_strided / opens_std, (opens_strided.shape[0],) + in_shape)
        v = np.reshape(volumes_strided / volumes_std, (opens_strided.shape[0], in_size))
        y = np.zeros((forward_values.shape[0], out_shape[0]))
        for i in range(y.shape[0]):
            y[i] = embed(
                forward_values[i] / forward_values_std,
                self.datainfo._y_min(),
                self.datainfo._y_max(),
                out_size,
            )
        n = np.arange(len(x))
        rnd = np.random.random(len(x))
        idx = n[
            (forward_values >= self.datainfo.y_std)
            | (forward_values <= -self.datainfo.y_std)
            
        ]
        x = x[idx]
        y = y[idx]
        v = v[idx]
        return x.astype("float32"), y.astype("float32")

In [4]:
    def load_dataset(self, tsv_file, count=0, skip=0):
        """Подготовка обучающей выборки (x,y). Тип x и y - numpy.ndarray.
        Аргументы:
        csv_file - файл с ценами формата csv с колонками: 'date','time','open','high','low','close','tickvol','vol','spread'
        count - количество используемых примеров датасета
        skip - сдвиг относительно конца файла
        Возврат:
        x, y - размеченные данные типа numpy.array, сохранные на диск файл dataset_file"""
        # stride не менять, должно быть =1
        stride = 1  # шаг "нарезки" входных данных
        in_shape = self.datainfo.input_shape
        out_shape = self.datainfo.output_shape
        future = self.datainfo.future
        in_size = self.datainfo._in_size()
        out_size = self.datainfo._out_size()  # out_shape[0]
        if not path.exists(tsv_file):
            print('Отсутствует файл "' + tsv_file + '"\nЗагрузка данных неуспешна')
            return None, None
        print("Чтение файла", tsv_file, "и создание обучающей выборки")
        data = pd.read_csv(
            tsv_file,
            sep="\t",
            header=0,
            dtype={
                "open": np.float32,
                "close": np.float32,
                "tickvol": np.float32,
                "vol": np.float32,
            },
            names=[
                "date",
                "time",
                "open",
                "high",
                "low",
                "close",
                "tickvol",
                "vol",
                "spread",
            ],
        )
        if skip == 0 and count == 0:
            open_rates = data["open"]
        elif skip == 0:
            open_rates = data["open"][-count:]
        elif count == 0:
            open_rates = data["open"][:-skip]
        else:
            open_rates = data["open"][-count - skip : -skip]
        d = np.diff(np.array(open_rates), n=1)
        x_std = d.std()
        self.datainfo.x_std = float(x_std)
        d = np.nan_to_num(
            d / x_std, posinf=self.datainfo._x_inf(), neginf=-self.datainfo._x_inf()
        )
        x_data = roll(d[:-future], in_size, stride)
        x_forward = roll(d[in_size:], future, stride)
        y_data = np.sum(x_forward * self.datainfo.x_std, axis=1)
        self.datainfo.y_std = float(y_data.std())
        x = np.reshape(x_data, (x_data.shape[0],) + in_shape)
        y = np.zeros((y_data.shape[0], out_shape[0]))

        for i in range(y.shape[0]):
            y[i] = embed(
                y_data[i], self.datainfo._y_min(), self.datainfo._y_max(), out_size,
            )
        n = np.arange(len(x))
        rnd = np.random.random(len(x))
        idx = n[
            (y_data >= self.datainfo.y_std)
            | (y_data <= -self.datainfo.y_std)            
        ]
        x = x[idx]
        y = y[idx]
        print(f"Загружено {len(x)} примеров")
        self.datainfo.save(self.name + ".cfg")
        return x.astype("float32"), y.astype("float32")

In [8]:
from predictor import Predictor
p = Predictor("models/37",
    input_shape=(4, 4, 4, 1),
    output_shape=(8,),
    predict_size=16,
    filters=256,
    kernel_size=2,
    dense_size=64)
x0, y0 = load_dataset(tsv_file="datas/EURUSD_M5_20000103_20200710.csv", count=1513200,skip=4608)
x1, y1 = load_dataset2(tsv_file="datas/EURUSD_M5_20000103_20200710.csv", count=1513200,skip=4608)

Отсутствует файл "datas/EURUSD_M5_20000103_20200710.csv"
Загрузка данных неуспешна


TypeError: load_dataset2() got an unexpected keyword argument 'tsv_file'